In [1]:
import numpy as np
import tensorflow.keras as keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.datasets import load_digits
from sklearn.utils import shuffle
from matplotlib import pyplot as plt
import tensorflow as tf
from scipy import stats as st
from keras.optimizer_v2.adam import Adam
import gc

Init Plugin
Init Graph Optimizer
Init Kernel


In [2]:
# Check for TensorFlow GPU access
print(tf.config.list_physical_devices())

# See TensorFlow version
print(tf.__version__)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
2.5.0


In [7]:
X_Train = np.genfromtxt('txt_files/test_x_norm.txt', delimiter=",")
Y_Train = np.genfromtxt('txt_files/test_y_continuous.txt', delimiter=",")

In [3]:
def create_k_fold_sets(arr, n=10):
    newarr = np.array_split(arr, n)
    splits = []
    for i in range(n):
        test_seg = newarr[i]
        if i == 0: 
            train_seg = newarr[1]
        else:
            train_seg = newarr[0]
        for j in range(n):
            if j == i:
                continue
            if not np.array_equal(train_seg, newarr[j]):
                train_seg = np.concatenate((train_seg, newarr[j]), axis=0)
        splits.append((train_seg, test_seg))
    return splits

In [ ]:
def to_final_class_vector(i):
    indice = np.where(i == np.amax(i))
    i2 = np.zeros(5)
    i2[indice[0][0]] = 1
    return i2

In [8]:
x_sets = create_k_fold_sets(X_Train)
y_sets = create_k_fold_sets(Y_Train)
len(x_sets)

10

In [4]:
def create_nn(learning_rate = 0.001, dropout_rate = None, regression = True):
    model = Sequential()
    model.add(Dense(10000, activation="relu", input_dim = 7))
    if dropout_rate is not None:
        model.add(Dropout(rate=dropout_rate))
    model.add(Dense(10000, activation="relu"))
    if dropout_rate is not None:
        model.add(Dropout(rate=dropout_rate))
    model.add(Dense(10000, activation="relu"))
    if dropout_rate is not None:
        model.add(Dropout(rate=dropout_rate))
    model.add(Dense(10000, activation="relu"))
    if dropout_rate is not None:
        model.add(Dropout(rate=dropout_rate))
    model.add(Dense(10000, activation="relu"))
    if dropout_rate is not None:
        model.add(Dropout(rate=dropout_rate))
    model.add(Dense(10000, activation="relu"))
    if dropout_rate is not None:
        model.add(Dropout(rate=dropout_rate))
    model.add(Dense(10000, activation="relu"))
    if dropout_rate is not None:
        model.add(Dropout(rate=dropout_rate))
    model.add(Dense(10000, activation="relu"))
    if dropout_rate is not None:
        model.add(Dropout(rate=dropout_rate))
    model.add(Dense(10000, activation="relu"))

    if regression is True:
        model.add(Dense(1, activation="linear"))
    else:
        model.add(Dense(5, activation="softmax"))

    optimizer = Adam(learning_rate = learning_rate)

    if regression is True:
        model.compile(optimizer=optimizer, loss= "mean_squared_error", metrics= ["MeanSquaredError"])
    else:
        model.compile(optimizer=optimizer, loss= "categorical_crossentropy", metrics= ["accuracy"])
        
    return model

In [5]:
def k_fold_train_nn(learning_rate, dropout_rate, regression, epochs):
    X_Train = np.genfromtxt('txt_files/test_x_norm.txt', delimiter=",")
    if regression is True:
        Y_Train = np.genfromtxt('txt_files/test_y_continuous.txt', delimiter=",")
        name = 'regression'
        w0 = open("saved_models/nn/regression/model_performances.txt", "a")
    else:
        Y_Train = np.genfromtxt('txt_files/test_y_classification.txt', delimiter=",")
        name = 'classification'
        w0 = open("saved_models/nn/classification/model_performances.txt", "a")
    x_sets = create_k_fold_sets(X_Train)
    y_sets = create_k_fold_sets(Y_Train)
    for i in range(len(x_sets)):

        model = create_nn(learning_rate, dropout_rate, regression)

        X_Train = x_sets[i][0]
        Y_Train = y_sets[i][0]
        X_Test = x_sets[i][1]
        Y_Test = y_sets[i][1]
        with tf.device('/device:GPU:0'):
            trained_model = model.fit(X_Train, Y_Train, batch_size=50, epochs=epochs, verbose=0, validation_split=0.0)
        acc = model.evaluate(X_Test, Y_Test)[1]
        w0.write(f'{name}_model_fold_{i} | test acc: {acc}')
        w0.write('\n')
        if regression is True:
            model.save(f'saved_models/nn/regression/{name}_model_fold_{i+1}.h5')
        else:
            model.save(f'saved_models/nn/classification/{name}_model_fold_{i+1}.h5')
        del model
        del trained_model
        gc.collect()
        keras.backend.clear_session()
    w0.close()

In [8]:
k_fold_train_nn(0.001, 0.1, True, 50)

Metal device set to: Apple M1 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2022-04-29 17:42:29.629925: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-29 17:42:29.630055: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-04-29 17:42:30.099237: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-04-29 17:42:30.099401: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-04-29 17:42:36.879638: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


3/7 [===========>..................] - ETA: 0s - loss: 1.8090 - mean_squared_error: 1.8090

2022-04-29 17:48:54.713296: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


7/7 [==============================] - 1s 38ms/step - loss: 1.5152 - mean_squared_error: 1.5152


...

2022-04-29 18:38:52.331780: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


3/7 [===========>..................] - ETA: 0s - loss: 0.7335 - mean_squared_error: 0.7335

2022-04-29 18:44:59.737185: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


7/7 [==============================] - 0s 38ms/step - loss: 0.8351 - mean_squared_error: 0.8351


<Figure size 432x288 with 0 Axes>

In [6]:
k_fold_train_nn(0.002, None, False, 50)

Metal device set to: Apple M1 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2022-04-29 18:47:30.625753: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-29 18:47:30.625881: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-04-29 18:47:31.098562: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-04-29 18:47:31.098762: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-04-29 18:47:37.615247: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


3/7 [===========>..................] - ETA: 0s - loss: 1.6711 - accuracy: 0.2396

2022-04-29 18:53:50.687336: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


7/7 [==============================] - 1s 40ms/step - loss: 1.5725 - accuracy: 0.2900


...

7/7 [==============================] - 0s 41ms/step - loss: 1.1177 - accuracy: 0.5150


2022-04-29 19:43:52.062385: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


2/7 [=======>......................] - ETA: 0s - loss: 1.2803 - accuracy: 0.4844

2022-04-29 19:49:56.707800: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


7/7 [==============================] - 0s 42ms/step - loss: 1.3220 - accuracy: 0.4250


In [ ]:
def ensemble_nn_prediction(models, regression, X):
    pred = None
    preds = []
    if regression:
        for model in models:
            predY = model.predict(X)
            preds.append(predY)
        avg = np.average(np.array(preds))
        pred = avg
    else:
        for model in models:
            predY = model.predict(X)
            preds.append(predY[0])
        preds = np.array(pred)
        pred = np.sum(preds, axis=0)
        pred = to_final_class_vector(pred)

    return pred